<a href="https://colab.research.google.com/github/campu1992/Emotios-detection-waitingRomm-AI/blob/main/Waiting_room_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

In [ ]:
# Cargar el conjunto de datos CIFAR-10 (que contiene imágenes etiquetadas)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Preprocesar las imágenes
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Definir la arquitectura del modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo
model.fit(x_train, y_train, epochs=80)

# Evaluar el modelo
model.evaluate(x_test, y_test)

# Guardar el modelo entrenado
model.save('modelo_clasificacion.h5')


Epoch 1/80
1563/1563 [==============================] - 7s 4ms/step - loss: 1.4336 - accuracy: 0.4872
Epoch 2/80
1563/1563 [==============================] - 6s 4ms/step - loss: 1.0985 - accuracy: 0.6148
Epoch 3/80
1563/1563 [==============================] - 6s 4ms/step - loss: 0.9754 - accuracy: 0.6608
Epoch 4/80
1563/1563 [==============================] - 5s 3ms/step - loss: 0.8906 - accuracy: 0.6904
Epoch 5/80
1563/1563 [==============================] - 6s 4ms/step - loss: 0.8274 - accuracy: 0.7115
Epoch 6/80
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7794 - accuracy: 0.7309
Epoch 7/80
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7244 - accuracy: 0.7478
Epoch 8/80
1563/1563 [==============================] - 5s 3ms/step - loss: 0.6852 - accuracy: 0.7600
Epoch 9/80
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6510 - accuracy: 0.7722
Epoch 10/80
1563/1563 [==============================] - 5s 3ms/step - loss: 0.615

Cargar el modelo de clasificación de emociones:

In [ ]:
# Cargar el modelo de clasificación de emociones entrenado
emotion_classifier = load_model('/content/modelo_clasificacion.h5')


 Definir la función de preprocesamiento de imágenes:

In [ ]:
def preprocess_image(image):
    # Convertir la imagen a escala de grises
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Redimensionar la imagen
    resized_image = cv2.resize(gray_image, (48, 48))

    # Normalizar la imagen
    normalized_image = resized_image / 255.0

    # Expandir la dimensión de la imagen (para que sea compatible con el modelo)
    expanded_image = np.expand_dims(normalized_image, axis=0)

    return expanded_image

Procesar y analizar el video:

In [ ]:
# Abrir el video
video_capture = cv2.VideoCapture('video.mp4')

while True:
    # Leer el siguiente fotograma del video
    ret, frame = video_capture.read()

    if not ret:
        break

    # Detectar y predecir emociones en el fotograma actual
    emotions = detect_and_predict_emotions(frame)

    # Visualizar las emociones predichas en el fotograma
    for (x, y, w, h), emotion in zip(faces, emotions):
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, emotion_labels[emotion], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Mostrar el fotograma con las emociones predichas
    cv2.imshow('Video Analysis', frame)

    # Presionar 'q' para salir
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cerrar la ventana y liberar recursos
cv2.destroyAllWindows()
video_capture.release()